In [6]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import ast
from pymongo import MongoClient
import urllib.parse
import time
from pymongo import MongoClient
import urllib.parse
import redis
import json

In [7]:
switch = True
def scrape():

    #get the data
    link = "https://www.blockchain.com/btc/unconfirmed-transactions"
    page = requests.get(link)
    soup = bs(page.content)

    numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
    hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})


    #Make the arrays for the values we need
    hash_value = [content.get_text() for content in hash]
    time_value = [content.get_text() for content in numbers[0::3]]
    BTC_value = [content.get_text() for content in numbers[1::3]]
    USD_value = [content.get_text() for content in numbers[2::3]]

    #Make dataframe
    dataFrame = {'_id' : hash_value, 'Time' : time_value, 'BTC-value' : BTC_value, 'USD-value' : USD_value}
    frame = pd.DataFrame(dataFrame)
    frame_complete = frame

    scrape.appendedDF = frame.append(frame_complete).drop_duplicates()
    frame = scrape.appendedDF
    return frame
    
while True:
    x = scrape()
    print(x)

                                                  _id   Time  \
0   fa23680b0ea9e9209d32932e9e03debe9008e14467058e...  13:10   
1   21074c15e56005b46ff4b2485eaec1c8e4ef75c3ed4168...  13:10   
2   8c2e578c6a77480d93f76f5ade95f74b7ef092d5579715...  13:10   
3   771667102886456eb2c124162dc012751814bf94fe2177...  13:10   
4   bf983ae1d187d9f6ac9d02d4f9eb1591d3865481b30254...  13:10   
5   ecf58ec393560469861c386d829ea07f27e0d1d162fe6c...  13:10   
6   f03f57dc2099c653637b1b3cb3651a09bd2e2c663241ee...  13:10   
7   9efddf5c3075cb1f253fa84ec86c02f29b2107638c4d79...  13:10   
8   43cd5b96586b9eeb9acc9cc99466e4123194b9b3c874a1...  13:10   
9   358c2538298d0ee5be8aceb077d7d88e26a2afbb553689...  13:10   
10  f74ea8564285ec3f3c8ad5e5d9fe5cc45f2522c6ad7d38...  13:10   
11  d1ce97dbc9bec855e4b446f3618897462c841905823f80...  13:10   
12  8953e19467b27473bb888c4094d4e4eb27d0d61f4658aa...  13:10   
13  937753849ec85da0409c7510177a2e070773cb90ece882...  13:10   
14  58839114c2a2d8011126547f03f2ea639524

                                                  _id   Time  \
0   fa23680b0ea9e9209d32932e9e03debe9008e14467058e...  13:10   
1   21074c15e56005b46ff4b2485eaec1c8e4ef75c3ed4168...  13:10   
2   8c2e578c6a77480d93f76f5ade95f74b7ef092d5579715...  13:10   
3   771667102886456eb2c124162dc012751814bf94fe2177...  13:10   
4   bf983ae1d187d9f6ac9d02d4f9eb1591d3865481b30254...  13:10   
5   ecf58ec393560469861c386d829ea07f27e0d1d162fe6c...  13:10   
6   f03f57dc2099c653637b1b3cb3651a09bd2e2c663241ee...  13:10   
7   9efddf5c3075cb1f253fa84ec86c02f29b2107638c4d79...  13:10   
8   43cd5b96586b9eeb9acc9cc99466e4123194b9b3c874a1...  13:10   
9   358c2538298d0ee5be8aceb077d7d88e26a2afbb553689...  13:10   
10  f74ea8564285ec3f3c8ad5e5d9fe5cc45f2522c6ad7d38...  13:10   
11  d1ce97dbc9bec855e4b446f3618897462c841905823f80...  13:10   
12  8953e19467b27473bb888c4094d4e4eb27d0d61f4658aa...  13:10   
13  937753849ec85da0409c7510177a2e070773cb90ece882...  13:10   
14  58839114c2a2d8011126547f03f2ea639524

                                                  _id   Time  \
0   8807652f4cb359747758fb573e4bbf07d286bb7a7a5703...  13:11   
1   de4b5527e97d3225739100f347393f0b08f8e472e66489...  13:11   
2   29d078fad6802a76ce6e2689e719a581edf114d6d70c58...  13:11   
3   66f2c44d729d4c9fefb57086a43a67ededefc5eb577f2f...  13:11   
4   78c1be4bdd79c5ffe662bc98c6b95093804c1fd195a7f2...  13:11   
5   cbb1e7b9ce8a30f505b612798f6ef55eb3ec2de13bcb15...  13:11   
6   0fd7413ecc54d262b949e99b7f6f6c0298eb048e3cf554...  13:11   
7   8ac0a78769f85dc9df52995b7d70aa3a21f8402abecdb4...  13:11   
8   7746bd60942d5e4b960fafb2faaa8f0f1f80c29e47d291...  13:11   
9   a379f902fc44b56958535cb3d6a7e1281ce8d48ccffada...  13:11   
10  35eaeca1b498b43a12f6b9207c119882ae0fd5420f4e00...  13:11   
11  6747d22dcc333e7dc7200c92a19cb3a3456489f096e8b2...  13:11   
12  4b33f98101a7cc7fc98768823731b61ed1a6752b83a3ae...  13:11   
13  20fc6a871b2863bf5773b0e14278cf640a69958e595e59...  13:11   
14  380cb7087335c90b971b2603f4168ba37c16

KeyboardInterrupt: 

In [8]:
df = x
#data cleaning
scrape.appendedDF['BTC-value'] = scrape.appendedDF['BTC-value'].map(lambda x: x.rstrip(' BTC'))
scrape.appendedDF['BTC-value'] = scrape.appendedDF['BTC-value'].str.replace(r'\,', '')

scrape.appendedDF['USD-value'] = scrape.appendedDF['USD-value'].map(lambda x: x.lstrip('$'))
scrape.appendedDF['USD-value'] = scrape.appendedDF['USD-value'].str.replace(r'\,', '')

scrape.appendedDF['USD-value'] = scrape.appendedDF['USD-value'].astype('float')
scrape.appendedDF['BTC-value'] = scrape.appendedDF['BTC-value'].astype('float')

In [9]:
r = redis.Redis()
r.flushdb()

True

In [ ]:
def cache():
    while 1:
        last_frame = df
        var = last_frame.to_dict(orient="records")
        for x in var:
            r.set(x["_id"], json.dumps(x), ex=200)
            
cache()

In [ ]:
# hashes_db = client["Hashes"]
# col_hashes = hashes_db["Hashes"]
# x = col_hashes.insert_one(y[0])